In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.append("..")
from utils.spark_config import create_spark_session
from src.data_loader import DataLoader
from src.eda_analyser import EdaAnalyser
from src.preprocessor import Preprocessor


In [4]:
spark = create_spark_session()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/24 15:36:16 WARN Utils: Your hostname, Jakubs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.157 instead (on interface en0)
25/12/24 15:36:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/24 15:36:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
data_loader = DataLoader(spark)
df = data_loader.load_raw_csv()

In [6]:
df.show(5)

+-----------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|            title|tag|   artist|year| views|            features|              lyrics| id|language_cld3|language_ft|language|
+-----------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|        Killa Cam|rap|  Cam'ron|2004|173166|{"Cam\\'ron","Ope...|[Chorus: Opera St...|  1|           en|         en|      en|
|       Can I Live|rap|    JAY-Z|1996|468624|                  {}|[Produced by Irv ...|  3|           en|         en|      en|
|Forgive Me Father|rap| Fabolous|2003|  4743|                  {}|Maybe cause I'm e...|  4|           en|         en|      en|
|     Down and Out|rap|  Cam'ron|2004|144404|{"Cam\\'ron","Kan...|[Produced by Kany...|  5|           en|         en|      en|
|           Fly In|rap|Lil Wayne|2005| 78271|                  {}|[Intro]\nSo they ...|  6|           en|      

In [5]:
import pyspark.sql.functions as F
df.filter(F.col('lyrics').isNull()).show(20)

+--------------------+--------------------+--------------------+----+-----+--------+------+----+-------------+-----------+--------+
|               title|                 tag|              artist|year|views|features|lyrics|  id|language_cld3|language_ft|language|
+--------------------+--------------------+--------------------+----+-----+--------+------+----+-------------+-----------+--------+
|           Killa Cam|           Killa Cam|                 Cam|NULL| NULL|    NULL|  NULL|NULL|         NULL|       NULL|    NULL|
|           Killa Cam|           Killa Cam|                NULL|NULL| NULL|    NULL|  NULL|NULL|         NULL|       NULL|    NULL|
|           Killa Cam|                 Cam|                NULL|NULL| NULL|    NULL|  NULL|NULL|         NULL|       NULL|    NULL|
|           Killa Cam|           Killa Cam|                 Cam|NULL| NULL|    NULL|  NULL|NULL|         NULL|       NULL|    NULL|
|Killa Killa Killa...|                NULL|                NULL|NULL| NULL| 

In [5]:
# df = Preprocessor.run(df)

In [6]:
df.limit(5).show()

+---------+----------+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
|    title|       tag| artist|year| views|       features|           lyrics|  id|language_cld3|language_ft|language|
+---------+----------+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
|Killa Cam|       rap|Cam'ron|2004|173166|"{""Cam\\'ron""|""Opera Steve""}"|NULL|         NULL|       NULL|    NULL|
|Killa Cam| Killa Cam|    Cam|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
|Killa Cam| Killa Cam|   NULL|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
|Killa Cam|       Cam|   NULL|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
|Killa Cam| Killa Cam|    Cam|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
+---------+----------+-------+----+------+---------------+------

In [7]:
analyser = EdaAnalyser(df)
analyser.run_full_eda_report(['title', 'lyrics', 'views'])

Columns and their datatypes present in the dataset:
root
 |-- title: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- features: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- language_cld3: string (nullable = true)
 |-- language_ft: string (nullable = true)
 |-- language: string (nullable = true)

Sample 5 rows:


+---------+----------+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
|    title|       tag| artist|year| views|       features|           lyrics|  id|language_cld3|language_ft|language|
+---------+----------+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
|Killa Cam|       rap|Cam'ron|2004|173166|"{""Cam\\'ron""|""Opera Steve""}"|NULL|         NULL|       NULL|    NULL|
|Killa Cam| Killa Cam|    Cam|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
|Killa Cam| Killa Cam|   NULL|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
|Killa Cam|       Cam|   NULL|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
|Killa Cam| Killa Cam|    Cam|NULL|  NULL|           NULL|             NULL|NULL|         NULL|       NULL|    NULL|
+---------+----------+-------+----+------+---------------+------

Dimension of the Dataframe is: (233187820, 11)
Number of null values: 


+-----+---------+---------+
|title|   lyrics|    views|
+-----+---------+---------+
| 4309|227019400|227965223|
+-----+---------+---------+

Top 5 most viewed pl songs:


+--------------------+-----------------+-----+
|               title|           artist|views|
+--------------------+-----------------+-----+
|Kiedy dzwonię gad...|           Dzakob| 9377|
| Top 10 albumów 2015|Rap Genius Polska| 9099|
| Top 10 albumów 2016|Rap Genius Polska| 8139|
|      Czarny koń WBW| Piotr Zwierzyski| 2988|
| Top 10 albumów 2017|Rap Genius Polska| 2706|
+--------------------+-----------------+-----+

Top 5 most viewed en songs:


+--------------------+-----------------+------+
|               title|           artist| views|
+--------------------+-----------------+------+
| Good Shit Copypasta|        Copypasta|329169|
|Dream About Teeth...|Your Subconscious|318423|
|Miracle on Ice Pr...|     Kurt Russell|242870|
|    UwU whats this ?|Reddit Copypastas|178311|
|      Jobs at Genius|           Genius|176911|
+--------------------+-----------------+------+



In [8]:
df.show(1)

+---------+---+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
|    title|tag| artist|year| views|       features|           lyrics|  id|language_cld3|language_ft|language|
+---------+---+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
|Killa Cam|rap|Cam'ron|2004|173166|"{""Cam\\'ron""|""Opera Steve""}"|NULL|         NULL|       NULL|    NULL|
+---------+---+-------+----+------+---------------+-----------------+----+-------------+-----------+--------+
only showing top 1 row
